## 1. Load main functions

In [46]:
from pprint import pprint
import pandas as pd
import json
import argparse

def generate_mapping_from_json(json_file, mapping_file):
    """_summary_
    """

    with open(json_file) as f:
        data = json.load(f)
    results = {}
    header = ''
    for prop in data['properties']:
        if data['properties'][prop]['type'] == 'object':
            if prop in data['required']:
                variables = data['properties'][prop]['properties']
                variables['record_id'] = {
                    'description': 'Record ID', 'type': 'autonumber'}
                for var in variables:
                    if 'enum' in variables[var]:
                        options = variables[var]['enum']
                        variables[var]['coding'] = '|'.join(
                            [f'{options.index(it)+1},{it}' for it in variables[var]['enum']])
                        variables[var]['enum'] = 'single_choice'
                    else:
                        variables[var]['coding'] = ''
                    for it in variables[var]:
                        if 'type' in variables[var]:
                            if 'string' in variables[var]['type']:
                                variables[var]['type'] = 'text'
                    study_format = ''
                    mapping = f'{var}='
                    if 'record_id' in var:
                        study_format = 'automated'
                    results[f'{list(variables.keys()).index(var)+1}'] = ['',
                                                                        '', '', study_format, var]+[val for val in variables[var].values()]
                    results[f'{list(variables.keys()).index(var)+1}'].append(mapping)
                    if header == '':
                        header = ['Study Variable Name', 'Study Variable Description', 'Study Variable Coding',
                                'Study Variable Format', 'NEW Variable Name']+list(variables[var].keys())
                        for h in range(len(header)):
                            if header[h] == 'type':
                                header[h] = 'NEW Variable Format'
                            if header[h] == 'description':
                                header[h] = 'NEW Variable Description'
                            if header[h] == 'coding':
                                header[h] = 'NEW Variable Coding'
                        header.append('NEW to Study Mapping')

    res = pd.DataFrame.from_dict(results, orient='index', columns=header)
    res.to_excel(f'{mapping_file}', index=None, header=True)


## 2. Datasets

In [48]:
json_file = "/Users/mamana/Projects/cvd/data/2022/19_05_22/cineca.minimal.json"
out_mapping_file = "/Users/mamana/Projects/cvd/data/2022/19_05_22/cineca.minimal.json.xlsx"

In [49]:
generate_mapping_from_json(json_file, out_mapping_file)